In [ ]:
# HOM_FTP_PATH = r"/home/ftp/ftp/dev/wsr-public/data_output"
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}
# bob.static {"token": ("98a517f5b880eeda7ed8568d7e8fb"
#                       "77834ac95284fede5c0459b5d2f8192"),
#             "path": os.path.join(HOM_FTP_PATH, "Confirmed.html"),
#             "filename": "../data_output/Confirmed.html"}
# bob.static {"token": ("7d0fe7cd6483514da087ccf755b574"
#                       "964d799167e7fa5a18962e5772b5c9"),
#             "path": "os.path.join(HOM_FTP_PATH, "Confirmed Rate.html"),
#             "filename": "../data_output/Confirmed Rate.html"}
# bob.static {"token": ("efd045150687448d5dbdc2c0acb3e5"
#                       "1b05e4ba28d2d29f4d533f29c14c2b"),
#             "path": "os.path.join(HOM_FTP_PATH, "Deaths.html"),
#             "filename": "../data_output/Deaths.html"}
# bob.static {"token": ("d14c73baee025b68caae42cb577dbe"
#                       "f7c665e74e0fa1b89458ea52a9aa3e"),
#             "path": ""os.path.join(HOM_FTP_PATH, "Fatality Rate.html",
#             "filename": "../data_output/Fatality Rate.html"}

In [ ]:
%run __init__.ipynb

In [ ]:
# --- Check if the data are already updated
FILEPATH = f'{OUTPUT_FOLDER}US_CONFIRMED_MAP.csv'
if os.path.exists(FILEPATH):
    DF_CHECK = pd.read_csv(FILEPATH, sep=';',
                           low_memory=False).drop('Unnamed: 0', axis=1)
    DATE_CHECK = DF_CHECK[(DF_CHECK['Date'] == (YESTERDAY.
                                                strftime('%m/%d/%y')))]
    if DATE_CHECK:
        print(f'Data already updated ! DateTime : {now}')
        %stop

In [ ]:
# --Create Dataframe
CUS = pd.read_csv(os.path.join(OUTPUT_FOLDER,
                               'US_CONFIRMED.csv'),
                  sep=";").drop('Unnamed: 0', axis=1)
COLS_TO_KEEP = ['UID', 'iso2', 'iso3', 'code3', ' FIPS',
                'Admin2', 'Lat', 'Long_', 'Combined_Key',
                'Province_State', 'Country_Region']
CUS = (CUS.
       melt(id_vars=COLS_TO_KEEP,
            value_name='Confirmed',
            var_name='Date').
       groupby(['Date', 'Province_State'], as_index=False).
       agg({'Confirmed': 'sum'}))

DUS = pd.read_csv(os.path.join(OUTPUT_FOLDER,
                               'US_DEATHS.csv'),
                  sep=";").drop('Unnamed: 0', axis=1)
COLS_TO_KEEP = ['UID', 'iso2', 'iso3', 'code3', 'FIPS',
                'Admin2', 'Lat', 'Long_', 'Combined_Key',
                'Province_State', 'Country_Region', 'Population']
DUS = (DUS.
       melt(id_vars=COLS_TO_KEEP,
            value_name='Deaths',
            var_name='Date').
       groupby(['Date', 'Province_State'], as_index=False).
       agg({'Deaths': 'sum', 'Population': 'sum'}))

DFUS = pd.merge(CUS, DUS,
                on=['Date', 'Province_State'],
                how='left').reset_index(drop=True)
DFUS.loc[(DFUS['Deaths'] != 0),
         'Fatality Rate'] = round((DFUS['Deaths'] /
                                   DFUS['Confirmed'] *
                                   100), 2)
DFUS.loc[(DFUS['Confirmed'] != 0),
         'Confirmed Rate'] = round((DFUS['Confirmed'] /
                                    DFUS['Population'] *
                                    100), 2)
DFUS.loc[(DFUS['Deaths'] != 0),
         'Deaths Rate'] = round((DFUS['Deaths'] /
                                 DFUS['Population'] *
                                 100), 2)

DFUS['Date'] = pd.to_datetime(DFUS['Date'],
                              format='%m/%d/%y').dt.strftime('%m/%d/%y')
DFUS_MAP = DFUS[(DFUS['Date'] == YESTERDAY.strftime('%m/%d/%y')) &
                (DFUS['Population'] != 0) &
                (DFUS['Confirmed'] != 0)]
if not DFUS_MAP:
    raise Exception(f'Data not updated ! DateTime : {NOW}')
else:
    DFUS_MAP.to_csv(os.path.join(OUTPUT_FOLDER,
                                 'US_CONFIRMED_MAP.csv'), sep=";")
DFUS_MAP.head()

In [ ]:
# Highlighted KPIS
def domain_108(dataf):
    """
    doc string
    """
    domain = dataf.copy()
    # Calcul US
    domain = (domain.
              groupby('Date', as_index=False).
              agg({'Confirmed': 'sum',
                   'Deaths': 'sum',
                   'Population': 'sum'}))
    domain.loc[(domain['Deaths'] != 0),
               'Fatality Rate'] = round((domain['Deaths'] /
                                         domain['Confirmed'] *
                                         100), 2)
    domain.loc[(domain['Confirmed'] != 0),
               'Confirmed Rate'] = round((domain['Confirmed'] /
                                          domain['Population'] *
                                          100), 2)
    domain.loc[(domain['Deaths'] != 0),
               'Deaths Rate'] = round((domain['Deaths'] /
                                       domain['Population'] *
                                       100), 2)

    # Create DF for HKPIS
    cols_to_keep = ['Date']
    domain = domain.melt(id_vars=cols_to_keep,
                         value_name='VALUE',
                         var_name='KPI')

    # Calc variation
    kpis = domain['KPI'].drop_duplicates().values.tolist()
    final_domain = pd.DataFrame()
    for kpi in kpis:
        tmp_df = (domain[domain['KPI'] == kpi].
                  sort_values(by="Date", ascending=False).
                  reset_index(drop=True))
        final_domain = final_domain.append(tmp_df)

    # Create df last + merge
    final_domain['D'] = (final_domain.index).astype(str)
    final_domain['D-1'] = (final_domain.index + 1).astype(str)
    cols_to_rename = {"D": "D-1", 'VALUE': 'VALUE_D-1'}
    df_last = final_domain[['D',
                            'KPI',
                            'VALUE']].rename(index=str,
                                             columns=cols_to_rename)
    final_domain = (final_domain.
                    merge(df_last, on=['D-1', 'KPI'], how='left').
                    drop(['D', 'D-1'], axis=1))

    # Add fields
    # SCENARIO
    final_domain['SCENARIO'] = (pd.to_datetime(final_domain['Date'],
                                               format='%m/%d/%y').
                                dt.strftime('%d/%m/%Y'))

    # VARIATION
    final_domain['VAR'] = (final_domain['VALUE'] -
                           final_domain['VALUE_D-1'])
    final_domain['VARP'] = (final_domain['VAR'] /
                            abs(final_domain['VALUE_D-1']) *
                            100)

    # UNIT & PRECISION
    final_domain.loc[:, 'UNIT_VALUE'] = ''
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate',
                     'UNIT_VALUE'] = ' %'
    final_domain.loc[:, 'UNIT_VAR'] = ''
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate',
                     'UNIT_VAR'] = ' pts'
    final_domain.loc[:, 'PRECISION_VALUE'] = ',.0f'
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate',
                     'PRECISION_VALUE'] = ',.2f'
    final_domain.loc[:, 'PRECISION_VAR'] = '+,.0f'
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate',
                     'PRECISION_VAR'] = '+,.3f'

    final_domain = (final_domain.
                    drop('Date', axis=1).
                    dropna().
                    reset_index(drop=True))
    return final_domain


DOMAIN108 = domain_108(DFUS)
bob.mongo.save_df(DOMAIN108, '108', DB_APP, True)
DOMAIN108

In [ ]:
def analysis(variable):
    """
    doc string
    """
    v_min = DFUS[variable].min()
    v_max = DFUS[variable].max()
    v_avr = round(DFUS[variable].mean(), 0)
    v_sum = DFUS[variable].sum()
    to_print = (f'{variable} : '
                f'Min: {v_min}, Max: {v_max}, '
                f'Average: {v_avr}, Sum: {v_sum}')
    return print(to_print)

In [ ]:
def update_chart_css(filename, css_filename):
    """
    doc string
    """
    my_map = None
    my_css = None
    with open(filename) as fil:
        my_map = fil.read()

    with open(css_filename) as fil:
        my_css = fil.read()
    if my_map.find('id="cs_css"') != -1:
        print("to do")
    else:
        result = my_map.replace("<body>",
                                ('<body><style '
                                 f'id="cs_css">{my_css}</style>'))
        with open(filename, "w") as fic:
            fic.write(result)

In [ ]:
def create_map(dataf, geo, kpi, scale, colors):
    """
    doc string
    """
    fig = px.choropleth_mapbox(dataf, geojson=geo, color=kpi,
                               locations="Province_State",
                               featureidkey="properties.NAME",
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron",
                               zoom=3,
                               center={"lat": 37.0902, "lon": -95.7129},
                               labels={'Province_State': 'State'})
    #                          animation_frame="Date",
    fig.update_layout(margin={"r": 0, "l": 0}, autosize=True)
    fig.write_html(OUTPUT_FOLDER + kpi + ".html")
    update_chart_css(f"{OUTPUT_FOLDER}{kpi}.html", f"{INPUT_FOLDER}map.css")
#     bob.makeStaticGettable(output_folder + kpi + ".html","dev")
#     return fig

In [ ]:
# Confirmed
KPI = 'Confirmed'
SCALE = (100, 250000)
COLORS = [[0.0, "#FFFFFF"],
          [0.01, "#B3D7D2"],
          [0.1, "#81B1BD"],
          [0.3, "#437F9D"],
          [1.0, "#00354E"]]
analysis(KPI)
create_map(DFUS_MAP, STATES20M, KPI, SCALE, COLORS)

In [ ]:
# Confirmed Rate
COLORS = [[0.0, "#FFFFFF"],
          [0.01, "#B3D7D2"],
          [0.1, "#81B1BD"],
          [0.3, "#437F9D"],
          [1.0, "#00354E"]]
SCALE = (0.1, 1)
KPI = 'Confirmed Rate'
analysis(KPI)
create_map(DFUS_MAP, STATES20M, KPI, SCALE, COLORS)

In [ ]:
# 'Deaths'
COLORS = [[0.0, "#FFFFFF"],
          [0.01, "#B3D7D2"],
          [0.1, "#81B1BD"],
          [0.3, "#437F9D"],
          [1.0, "#00354E"]]
SCALE = (0, 10000)
KPI = 'Deaths'
analysis(KPI)
create_map(DFUS_MAP, STATES20M, KPI, SCALE, COLORS)

In [ ]:
# Fatality Rate
COLORS = [[0.0, "#FFFFFF"],
          [0.01, "#B3D7D2"],
          [0.1, "#81B1BD"],
          [0.3, "#437F9D"],
          [1.0, "#00354E"]]
SCALE = (0, 10)
KPI = 'Fatality Rate'
analysis(KPI)
create_map(DFUS_MAP, STATES20M, KPI, SCALE, COLORS)

In [ ]:
# html
RACINE = 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'
CONFIRMED = [{'URL': (RACINE +
                      ('98a517f5b880eeda7ed8568d7e8fb7'
                       '7834ac95284fede5c0459b5d2f8192')),
              'KPI': 'Confirmed'}]
CONFIRMEDRATE = [{'URL': (RACINE +
                          ('7d0fe7cd6483514da087ccf755b574'
                           '964d799167e7fa5a18962e5772b5c9')),
                  'KPI': 'Confirmed Rate'}]
DEATHS = [{'URL': (RACINE +
                   ('efd045150687448d5dbdc2c0acb3e5'
                    '1b05e4ba28d2d29f4d533f29c14c2b')),
           'KPI': 'Deaths'}]
FATALITYRATE = [{'URL': (RACINE +
                         ('d14c73baee025b68caae42cb577dbe'
                          'f7c665e74e0fa1b89458ea52a9aa3e')),
                 'KPI': 'Fatality Rate'}]

LIST_KPIS = CONFIRMED + CONFIRMEDRATE + DEATHS + FATALITYRATE
DOMAIN = pd.DataFrame(LIST_KPIS)
DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                          DOMAIN['URL'] +
                          ("' frameborder='0' style='overflow:hidden;"
                           "height:85vh;width:100%' height='100%' "
                           "width='100%'></iframe> <style>.modebar"
                           "{display: none!important;}</style>"))
if USE_MONGO:
    bob.mongo.save_df(DOMAIN, '109', DB_APP, True)
DOMAIN